1/3 cL^2/(ne)**2*2
2°, 2/3 cL^2/ ne^2.2 + 

In [1]:
from scipy.integrate import quad
import numpy as np
import matplotlib.pyplot as plt  
import math   
import random   
import copy
import MEF
from scipy.integrate import quad

ModuleNotFoundError: No module named 'MEF'

<div style="background:#999900">no tengo tu módulo MEF</div>

In [ ]:
Ne= 1 #Numero de elementos en los que divido la barra
Nn= Ne+1 #numero de nodos
E= 30E6
T = np.array([-10, 0])
L0= 0.6
a1= -1/L0
a0= 1
b1= 1/L0
b0= 0
A= 2*E-4
glxn= 1
#puedo poner las funciones interpolacion como una matriza
N = np.array([(a1, a0),(b1 , b0)]) #funciones de interpolacion,, coeficientes, 1°grado

In [ ]:
MN= np.linspace(0,L0,Nn) #posicion de los nodos
MC= np.array([[i,i+1] for i in range (Ne)])
k_elem= {}

In [ ]:
def carga_axial_centrada(T,i,MC,MN):
    T[1] = -T[0]*MN[MC[i][0]]
    return T

In [ ]:
def derivada_inter(N):
    #N es el vector con los coeficientes de la funcion interpolacion
    # N= an*x^n+ ....+ a2*x^2+ a1*x+a0, length -1=grado pol  
    #N' = (gr-1-i)*an
    gr= len(N) -1
    N_deriv= []
    for i in range(len(N)-1):
        N_deriv.append((gr-i)*N[i])
    return N_deriv

Traté de generalizar la obtencion de la matriz B, para despues obtener la k elemental de cada elemento.
A la funcion le ingreso como parametro las dimensiones de la barra y el modulo de young, la cantidad de elementos, la matriz de conectividad y de nodos.
La generalizacion era mas por el caso de que si no tengo las funciones de interpolacion que sean polinomios de primer orden porder hacerlas (no pude tampoco con eso, porque generé el vector B con las derivadas de las funciones interpolacion, pero no llegue a calcular la matriz de esa forma. no sabia como hacer B^T*B para obtener la matriz.
Entonces, en la funcion, leo la cantidad de funciones interpolacion que tiene el sistema. Creo la matriz B de zeros, que podria ser directamente k_elem.
Despues genero un for para calcular ir recorriendo todas las filas de la matriz B y otro for para recorrer las columnas de la matriz. Calculó con la funcion derivada_inter la derivada para cada polinomio de interpolacion de lo que seria el vector B^T y luego del vector B. Y los agrego en la matriz a partir de la posicion que le corresponde segun los indices i,j.

<div style="background:#999900"> ese nivel de generalización escapa al scope de la materia. En general trabajamos con matrices elementales fijas. </div>

In [ ]:
def k_elem_distribucion(MC,MN,E,A,i,L0,N,Ne):
    cant_func = N.shape[1] #cantidad de funciones de interpolacion
    B= np.zeros([cant_func,cant_func])
    for i in range(cant_func):
        for j in range(cant_func):          
            N_deriv1= derivada_inter(N[i]) #es lineal, me da un solo elemento
            N_deriv2= derivada_inter(N[j])
            B[i,j]= N_deriv1[0]*N_deriv2[0] #corregir para el caso de que no sea linea
    ke= E*A/(L0/Ne)
    K_elem= ke*B
    K_elem[np.abs(K_elem/K_elem.max()) < 1e-15] = 0

    return K_elem

In [ ]:
Kglobal= np.zeros([Nn*glxn,Nn*glxn])

In [ ]:
for e in range(Ne):
    k_elem[e]= k_elem_distribucion(MC,MN,E,A,e,L0,N,Ne)
    fe = np.abs(k_elem[e].max()) #factor de escala
    pos= np.array([e,e+1])
    #with open(archivo,'a') as f:
     #   f.write(f'\nelemento {e}, fe ={fe:4e}\n')
      #  f.write(f'{k_elem[e]/fe}\n')
    Kglobal[np.ix_(pos,pos)] += k_elem[e]

In [ ]:
print(Kglobal)

    # Nnxe es siempre dos, el elemento queda definido a partir de dos nodos.
    i in range(2):
        rangoi= np.linspace(i*glxn,(i+1)*glxn,2, dtype = "int")
        rangoni= np.linspace(MC[e][i]*glxn,(MC[e][i]+1)*glxn,2, dtype = "int")
        for j in range(2):
            rangoj= np.linspace(j*glxn,(j+1)*glxn,2, dtype = "int")
            rangonj= np.linspace(MC[e][j]*glxn,(MC[e][j]+1)*glxn,2, dtype = "int")
            Kglobal[np.ix_(rangoni,rangonj)] += k_elem[e][np.ix_(rangoi,rangoj)]

In [ ]:
def int_polinomio(p):
    gr= len(p)-1
    p_int= []
    for i in range(len(p)-1):
        p_int.append(p[i]/(gr+1))
    p_int.append(0)
    return p_int

<div style="background:#999900">el problema de integrar así sería que te falta el termino cte que cale de integrar el polinomio, y a demas estas son tegrales definidas. quizá te convenga usar directamente la función `np.integrate.quad`</div>

In [ ]:
#el vector fuerzas no lo puedo armar, no encuentro la forma para integrar, o armar la funcion N1*T(x) e intgrarla
fuerzas = np.zeros([1,Nn])
for i in range(Nn):
    print(i)
    for j in range(len(N)): #para la cantidad de funciones de interpolacion
        integrate1= int_polinomio(N[j,:])
        fuerzas[i]=np.polyval(T,MN[i])
    if i == 0:
        #fuerzas[i]=(MN[MC[i][1]]-MN[MC[i][0]])*np.polyval(T,MN[MC[i][1]])
        fuerzas[i]=(MN[i+1]-MN[i])*np.polyval(T,MN[i])
    else:
        #T= carga_axial_centrada(T,i,MC,MN) #calculo la carga axial aplicada centrada en el punto de la barra en el que estoy
        #auxiliar= MC[i][1]
        #h0= np.polyval(T,MN[MC[i][0]]) #altura del rectangulo
        h0= np.polyval(T,MN[i-1])
        base=MN[i]-MN[i-1]
        fuerzas[0,i]= (base*(np.polyval(T,MN[i])-h0))+base*h0

In [ ]:
fuerzas

In [ ]:
#me falta armar el s,r,Us,Fr
r= np.array([0])
s= np.array([1])
Us= np.array([0])
Fr= np.array([fuerzas[0]])

In [ ]:
U,F = MEF.solve1D(K_global,s,r,Us,Fr)